<a href="https://colab.research.google.com/github/glamacles/nn_sliding_law/blob/kevin-NN/turner_nn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# get the sliding law and data from github
! git clone https://github.com/glamacles/nn_sliding_law

Cloning into 'nn_sliding_law'...
remote: Enumerating objects: 104, done.
remote: Counting objects: 100% (104/104), done.
remote: Compressing objects: 100% (77/77), done.
remote: Total 104 (delta 19), reused 99 (delta 17), pack-reused 0
Receiving objects: 100% (104/104), 4.09 MiB | 13.91 MiB/s, done.
Resolving deltas: 100% (19/19), done.


In [2]:
# import firedrake
try:
    import firedrake
except ImportError:
    !wget "https://fem-on-colab.github.io/releases/firedrake-install-real.sh" -O "/tmp/firedrake-install.sh" && bash "/tmp/firedrake-install.sh"
    import firedrake

--2024-06-22 16:08:14--  https://fem-on-colab.github.io/releases/firedrake-install-real.sh
Resolving fem-on-colab.github.io (fem-on-colab.github.io)... 185.199.110.153, 185.199.108.153, 185.199.111.153, ...
Connecting to fem-on-colab.github.io (fem-on-colab.github.io)|185.199.110.153|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4581 (4.5K) [application/x-sh]
Saving to: ‘/tmp/firedrake-install.sh’

/tmp/firedrake-inst 100%[===================>]   4.47K  --.-KB/s    in 0s      

2024-06-22 16:08:14 (41.3 MB/s) - ‘/tmp/firedrake-install.sh’ saved [4581/4581]

+ INSTALL_PREFIX=/usr/local
++ echo /usr/local
++ awk -F/ '{print NF-1}'
+ INSTALL_PREFIX_DEPTH=2
+ PROJECT_NAME=fem-on-colab
+ SHARE_PREFIX=/usr/local/share/fem-on-colab
+ FIREDRAKE_INSTALLED=/usr/local/share/fem-on-colab/firedrake.installed
+ [[ ! -f /usr/local/share/fem-on-colab/firedrake.installed ]]
+ PYBIND11_INSTALL_SCRIPT_PATH=https://github.com/fem-on-colab/fem-on-colab.github.io/raw/9be49f2/rele

In [3]:
# move to the directory
cd nn_sliding_law/examples/turner/inference

/content/nn_sliding_law/examples/turner/inference


In [5]:
# run the python script
run nn_law

  v_avg = np.nanmean(velocities,axis=0)

  warn(



0 0.004465970894620247 169.90583276748657
1 0.011149587241320287 0.2544546127319336
2 0.03235179563608277 0.23598241806030273
3 0.045609870655273534 0.19910478591918945
4 0.050253974352457736 0.18471169471740723
5 0.052756987034599204 0.18620610237121582
6 0.05359060896970588 0.18576693534851074
7 0.052257325154749915 0.17773962020874023
8 0.04871085433356698 0.17226672172546387
9 0.04363107814556677 0.17483925819396973
10 0.037955243543048856 0.17004036903381348
11 0.032293800335886 0.17210888862609863
12 0.027007751974357563 0.17896294593811035
13 0.02228465418887504 0.16816186904907227
14 0.01819421692558043 0.17583417892456055
15 0.01473189221608967 0.2317183017730713
16 0.01185105467930253 0.22377753257751465
17 0.009485031887948026 0.21170473098754883
18 0.007561084211053115 0.21548748016357422
19 0.006008568623058319 0.1849048137664795
20 0.004763186368857086 0.23299694061279297
21 0.003768759409683742 0.2514455318450928
22 0.0029775451730876257 0.25703001022338867
23 0.00234975

  warn(



11.618489464867553


Building the neural network.

In [6]:
# import nn dependencies
import torch
import torch.nn as nn
import torch.nn.functional as F

In [7]:
# define the class, layers, and forward pass
class MLP(nn.Module):
  '''
    Multilayer Perceptron.
  '''
  def __init__(self):
    super().__init__()
    self.layers = nn.Sequential(
      nn.Linear(4, 32),     # number of features, number of desired hidden layers
      nn.ReLU(),
      nn.Linear(32, 1)      # number of hidden layers, number of outputs
    )


  def forward(self, x):   # x = input
    '''Forward pass'''
    return self.layers(x)

In [20]:
# Initialize the MLP and run
mlp = MLP()   # initialize
log_beta = mlp(features) # feed the model with the features
beta = torch.exp  # take the exp of the NN output to ensure positivity